# **Module 1: Introduction to Spatial Data Analysis in Python**
## **Prerequisites**

### Data
In this example, we will use 3 datasets from the Minnesota Geospatial Commons, which have been downloaded, cleaned, transformed and saved to the directory `./data-module-1/` for this workshop.
- `soil_samp_2021-10-05.csv` - Minnesota Six-Inch Soil Temperature: https://gisdata.mn.gov/dataset/geos-soil-temp-network
- `gw_provinces_extra.shp` - Groundwater Provinces of Minnesota 2021: https://gisdata.mn.gov/dataset/geos-groundwater-provinces-mn
- `cdl3_3km.tif` - Cropland Data Layer 2020, Minnesota: https://gisdata.mn.gov/dataset/agri-cropland-data-layer-2020      

### Software
To execute the code you will need a `conda` environment for Python with the packages imported below.

In [ ]:
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import rasterio
import numpy as np
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from rasterio.crs import CRS
from rasterio.plot import plotting_extent
sns.set_style("whitegrid")

## **Working with Spatial Points Data in Python**
### Importing XY data

Our first step is to read in the `.csv` file that contains our attribute data with geographic coordinates. For this example, we will be using the *Minnesota Six-Inch Soil Temperature* data. 

> The Minnesota Department of Agriculture (MDA) Six-Inch Soil Temperature Network provides real time soil temperatures at locations across Minnesota. The network was established to assist in following best management practices for fall nitrogen fertilizer application, which refer to a soil temperature at a six-inch depth.

For today's workshop, a random October 5, 2021 temperature has been added for illustrative purposes. We read in `.csv` data using the Python `pandas` package, which enables operations on tabular data (data frames).

In [ ]:
soil_samp_df = pd.read_csv("./data-module-1/soil_samp_2021-10-05.csv")
soil_samp_df

### Convert `DataFrame` to `GeoDataFrame`
In this step we use the spatial information from `x` and `y` columns to convert the initial table `DataFrame` into a `GeoDataFrame` to enable operations for spatial analysis of the features. Because coordinates are supplied in degrees of latitude and longitude, we need to set World Geodetic System 1984 (`epsg:4326`) as a coordinate system.

In [ ]:
soil_samp_gdf = gpd.GeoDataFrame(soil_samp_df, geometry=gpd.points_from_xy(soil_samp_df.x, soil_samp_df.y))
soil_samp_gdf.set_crs("epsg:4326", inplace=True)
soil_samp_gdf.drop(columns=["x", "y"], inplace=True)
soil_samp_gdf

### Summarize Point Data

In [ ]:
print ("First 5 rows of the GeoDataFrame: ")
soil_samp_gdf.head()

In [ ]:
print ("Extent of the data is defined by the following bounding box: ")
soil_samp_gdf.total_bounds

In [ ]:
print ("The following contains the information on the Coordinate Reference System: ")
soil_samp_gdf.crs

In [ ]:
print ("Summarize numerical fields")
soil_samp_gdf.describe()

In [ ]:
print ("Describe string columns")
soil_samp_gdf.describe(include=[object])

### Vizualize Point Data

In [ ]:
print ("Simple point map")
soil_samp_gdf.plot()

**Although a simple plot can be easily produced with Python, it is rarely the final product that allows you to explore and share insights about the data. The examples below provide options on creating a interactive map and on how you can better control figure aesthetics and add important functional components, such as legend.**

In [ ]:
print ("Interactive map")
soil_samp_gdf.explore()

In [ ]:
print ("Display categorical values on the plot")
fig, ax = plt.subplots(figsize=(14,8))
soil_samp_gdf.plot(ax=ax, column="source", legend=True, cmap="tab20",  categorical=True)
ax.set_title("Minnesota Six-Inch Soil Temperature source", weight="bold")

**You can find more information on suported colors and colormaps in `matplotlib` by going to the following links:**
- https://matplotlib.org/stable/tutorials/colors/colormaps.html
- https://matplotlib.org/stable/gallery/color/named_colors.html

In [ ]:
print ("Display continuous values on the plot")
fig, axs = plt.subplots(1,2, figsize=(14,8))
soil_samp_gdf.plot(ax=axs[0], column="temp_20211005", legend=True, cmap="coolwarm")
axs[0].set_title("Minnesota Six-Inch Soil Temperature, degrees F", weight="bold")
soil_samp_gdf.plot(ax=axs[1], column="temp_20211005", legend=True, scheme="User_Defined", cmap="YlOrBr", 
                   classification_kwds=dict(bins=[40,45,50,55,60]), 
                   legend_kwds={"labels": ["< 40", "40 - 45", "45 - 50", "50 - 55", "55 - 60", "> 60"]})
axs[1].set_title("Minnesota Six-Inch Soil Temperature, degrees F", weight="bold")
plt.tight_layout()

## **Working with Polygon Data in Python**
### Importing Shapefiles

Our first step is to read in the `.shp` file that contains our attribute data and geometry data. For this example, we will be using the *Groundwater Provinces of Minnesota 2021* data. 

> The Minnesota Groundwater Provinces map summarizes aquifer and groundwater resource differences at the regional level. Some parts of the state have several groundwater resources to choose from, while other parts of the state may have only limited groundwater resources available.

For today's workshop, 3 random variables have been added as fields for illustrative purposes. We read in vector data using the `geopandas` package directly.

In [ ]:
gw_prov_gdf = gpd.read_file("./data-module-1/gw_provinces_extra.shp")

### Summarize Polygon Data

In [ ]:
print ("First 5 rows of the GeoDataFrame: ")
gw_prov_gdf.head()

In [ ]:
print ("Extent of the data is defined by the following bounding box: ")
gw_prov_gdf.total_bounds

In [ ]:
print ("The following contains the information on the Coordinate Reference System: ")
gw_prov_gdf.crs

In [ ]:
print ("Summarize numerical fields")
gw_prov_gdf.describe()

In [ ]:
print ("Describe string columns")
gw_prov_gdf.describe(include=[object])

### Vizualize Polygon Data

In [ ]:
print ("Simple polygon map")
gw_prov_gdf.plot()

In [ ]:
print ("Interactive map")
gw_prov_gdf.explore()

In [ ]:
print ("Display categorical values on the plot")
fig, axs = plt.subplots(1,3, figsize=(14,7))
gw_prov_gdf.plot(ax=axs[0], column="PROVINCE", legend=True, cmap="Paired")
axs[0].set_title("Groundwater Provinces of Minnesota PROVINCE", weight="bold")
gw_prov_gdf.plot(ax=axs[1], column="var1", legend=True, cmap="plasma")
axs[1].set_title("Groundwater Provinces of Minnesota var1", weight="bold")
gw_prov_gdf.plot(ax=axs[2], column="var2", legend=True, cmap="Set3")
axs[2].set_title("Groundwater Provinces of Minnesota var2", weight="bold")
plt.tight_layout()

In [ ]:
print ("Display continuous values on the plot")
fig, axs = plt.subplots(1,2, figsize=(14,6))
gw_prov_gdf.plot(ax=axs[0], column="var3", legend=True, cmap="viridis_r")
axs[0].set_title("Groundwater Provinces of Minnesota var3", weight="bold")
gw_prov_gdf.plot(ax=axs[1], column="var3", legend=True, scheme="User_Defined", cmap="YlOrBr", 
                 classification_kwds=dict(bins=[100,150,200,250,300,350]),
                 legend_kwds={"labels": ["< 100", "100 - 150", "150 - 200", "200 - 250", 
                                         "250 - 300", "300 - 350", "> 360"]})
axs[1].set_title("Groundwater Provinces of Minnesota var3", weight="bold")
plt.tight_layout()

## **Working with Raster Data in Python**
### Import Raster Files

Our first step is to read in the raster file that contains our data. For this example, we will be using the *Cropland Data Layer 2020, Minnesota* data. 

> The United States Department of Agriculture (USDA), National Agricultural Statistics Service (NASS) Cropland Data Layer (CDL) is a raster, geo-referenced, crop-specific land cover data layer.

For today's workshop, the data has been transformed to represent 3 categories: corn (0), other cropland (1), and non cropland (2). We read in raster data using the `rasterio` package.

In [ ]:
cdl_dataset =  rasterio.open("./data-module-1/cdl3_3km.tif")

### Summarize raster dataset geoproperties 

In [ ]:
print ("Number of rows is equal to {}".format(cdl_dataset.height))
print ("Number of columns is equal to {}".format(cdl_dataset.width))
print ("Extent of the dataset: {}".format(cdl_dataset.bounds))
print ("Coordinate Reference System: {}".format(cdl_dataset.crs))

### Summarize raster data values

In [ ]:
cdl_array = cdl_dataset.read(1)
cdl_array = cdl_array.astype(float)
cdl_array[cdl_array==cdl_dataset.nodata] = np.nan
print ("Raster unique values are: {}".format(np.unique(cdl_array)))

In [ ]:
plt.hist(cdl_array.flatten(), facecolor="grey", alpha=0.75)
plt.title("Minnesota CDL Land Cover Histogram")
plt.ylabel("Pixel count")
plt.xticks([0,1,2], labels=["corn", "other crops", "non cropland"])
plt.xlim(0,2)

### Visualize raster data

In [ ]:
clrs = ["gold", "mediumseagreen", "thistle"]
labels = ["corn", "other cropland", "non cropland"]
cmap = ListedColormap(clrs)
norm = colors.BoundaryNorm([1,2], 3)
plt.figure(figsize = (14,8))
plt.imshow(cdl_array, interpolation="none", cmap=cmap, norm=norm)
plt.title("Minnesota CDL", weight="bold")
c = [ mpatches.Patch(facecolor=clrs[i]) for i in range(len(labels))]
plt.legend(c, labels)

## **Layering Features**
To understand better landscape patterns, spatial distribution of features and their interactions, it is often useful to display multiple layers on the same map. To ensure that the layers align, follow these rules:
- all layers need to be in the same coordinate system (here we choose the coordinate system of the land cover raster layer);
- axis lables for raster data need to be updated to display a spatial extent of interest instead of the pixel counts.

In [ ]:
fig, ax = plt.subplots(figsize = (14,8))
plot_extent = plotting_extent(cdl_array, cdl_dataset.transform)
clrs = ["khaki", "darkseagreen", "lavender"]
labels = ["corn", "other cropland", "non cropland"]
cmap = ListedColormap(clrs)
norm = colors.BoundaryNorm([1,2], 3)
ax.imshow(cdl_array, interpolation="none", cmap=cmap, norm=norm, extent=plot_extent)
c = [mpatches.Patch(facecolor=clrs[i]) for i in range(len(labels))]
leg1 = plt.legend(c, labels, bbox_to_anchor=(1.3, 1), title="Land Cover")
gw_prov_gdf.to_crs(cdl_dataset.crs).plot(ax=ax, facecolor="none", edgecolor="grey")
soil_samp_gdf.to_crs(cdl_dataset.crs).plot(ax=ax, column="temp_20211005", legend=True, scheme="User_Defined", 
                                           cmap="coolwarm",  edgecolor="grey", markersize=50,
                                           classification_kwds=dict(bins=[40,45,50,55,60]), 
                                           legend_kwds={"labels": ["< 40", "40 - 45", "45 - 50", 
                                                                   "50 - 55", "55 - 60", "> 60"],
                                                       "bbox_to_anchor":(1.37, 0.85),
                                                        "title":"Soil Temperatue, degrees F"})
plt.title("Minnesota Land Cover and Soil Temperature", weight="bold")
plt.gca().add_artist(leg1)

### Export final map to a file

In [ ]:
fig.savefig("./data-module-1/MN_final_map.jpg", bbox_inches='tight', dpi=150)

## **Exercises**

### Data
-  `ea_geo.csv` - Malawi Living Standard Measurement Survey Integrated Household Sample (LSMS-IHS) point data are available from https://microdata.worldbank.org/index.php/catalog/3818  
- `mwi_lsms.shp` - Malawi subnational divisions can be downloaded from https://data.humdata.org/dataset/malawi-administrative-level-0-3-boundaries
- `MWI_msk_alt.vrt` - Malawi Digital Elevation Model (DEM) is derived from NASA's Shuttle Radar Topography Mission data product. 

For today's workshop, the data has been downloaded, cleaned, transformed, and saved to the directory `./data-module-1/`.

**Question 1. Read and display the head of the `ea_geo.csv` file stored under `./data-module-1/`.**

**Question 2. Which columns store the geographic coordinates? Transform `DataFrame` into `GeoDataFrame` by using these columns. Display the head of the `GeoDataFrame`.**

**Question 3. Create an interactive map to ensure that the data have been properly geocoded.**

**Question 4. Create a static map with 2 subplots: based on columns `dist_road` and `dist_border`. Display values as continuous.**

**Question 5. Read the `mwi_lsms.shp` shapefile located in the `./data-module-1/` folder. Display the first 2 records of `GeoDataFrame`.**

**Question 6. Explore the characteristics of the `GeoDataFrame`.**  
- How many columns does it have?
- How many rows does it have?
- Calculate summary statistics of the numerical fields.

**Question 7. Create an interactive map of the `GeoDataFrame`.**

**Question 8. Create a static map with 2 subplots: based on columns `croplnd` and `poverty`. Display values as continuous.**

**Question 9. Read the Malawi DEM raster file stored as `MWI_msk_alt.vrt`. Convert the data type to float and reset `NoData` values to `np.nan`.**

**Question 10. Explore the characteristics of your raster.**
- How many rows and columns does it have?
- What is the spatial extent of the dataset?
- What is the coordinate reference system?
- Plot a histogram to display the distribution of values.

**Question 11. Plot Malawi DEM array. Use `terrain` as a `cmap` option.**

**Question 12. Display multiple features on the same map:**
- add Malawi DEM (use the coorditate system from this dataset as a reference for others), use `terrain` as `cmap` option;
- add Malawi district boundaries (polygon), display only the edges with `grey` color;
- add Malawi LSMS points, display `dist_agmrkt` column, include the legend, and keep the default `cmap`.